In [1]:
% load_ext autoreload
% autoreload 2
% matplotlib inline

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import sys
import os

In [3]:
# add gcnn module
sys.path.append('..')

# bigger figure
plt.rcParams['figure.figsize'] = 18, 5

In [4]:
# setting seed for reproducability
np.random.seed(0)
torch.manual_seed(0);

In [5]:
# scope to specfic gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [6]:
from gcnn.datasets import load_icebergs

# train = all labelled cases from Kaggle
measures = load_icebergs('train')
measures.head(4)

,band_1,band_2,inc_angle,is_iceberg
id,,,,
dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train, test = train_test_split(range(len(measures)), test_size=0.15, stratify=measures.is_iceberg, random_state=0)

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
band_scaler = MinMaxScaler()
angle_scaler = MinMaxScaler()

In [11]:
for _, e in measures.iloc[train].iterrows():
    band_scaler.partial_fit(e.band_1.reshape(1, -1))
    band_scaler.partial_fit(e.band_2.reshape(1, -1))
    
angle_scaler.fit(measures.iloc[train].inc_angle.dropna().values.reshape(-1, 1));

Usable data contains the two bands, the angle and the target (label).

In [12]:
angles = angle_scaler.transform(measures.inc_angle.fillna(0).values.reshape(-1, 1))

In [13]:
targets = measures.is_iceberg.values.reshape(-1, 1)

In [14]:
from gcnn import graph

In [15]:
# commented as slow [nx.average_node_connectivity(g) for g in small_graphs]

In [16]:
from gcnn.coarsening import graclus

In [17]:
scores = pd.DataFrame([], columns=['name', 'accuracy', 'precision', 'recall', 'f1']).set_index('name')
scores

,accuracy,precision,recall,f1
name,,,,


In [18]:
bands = np.stack([
    band_scaler.transform(np.stack(measures.band_1)),
    band_scaler.transform(np.stack(measures.band_2)),
], axis=1).reshape(-1, 2, 75, 75)

In [19]:
flat_features = np.c_[bands.reshape(-1, 2 * 75 * 75), angles.reshape(-1, 1)]
flat_features.shape

(1604, 11251)

In [20]:
from gcnn.utils import score_classification

In [21]:
cuda = torch.cuda.is_available()
cuda

True

In [22]:
import torch.utils.data as data_utils

Prepare pytorch tensors.

In [23]:
from gcnn.nets import BaselineCNNSimple
from gcnn.utils import NNplusplus
from skorch import NeuralNet

Skorch provides a sklearn interface over pytorch models.

In [24]:
cnn = NNplusplus(
    BaselineCNNSimple,
    use_cuda=cuda,
    optimizer=torch.optim.Adam,
    criterion=nn.BCEWithLogitsLoss, 
    batch_size=50, # 25 # 50
    max_epochs=30, 
    lr=0.001, # 
)

In [25]:
features = bands[train].astype(np.float32)
labels = targets[train].astype(np.float32)

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
param_dist_cnn = {
    'batch_size': [25,50],
    'lr':[0.001,0.0001,0.00001],
}

In [28]:
random_search_cnn = GridSearchCV(cnn,param_dist_cnn)

In [29]:
random_search_cnn.fit(features, labels)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.6926        0.6922  1.4109
      2        0.6929        0.6914  0.5855
      3        0.6897        0.6918  0.4630
      4        0.6900        0.6905  0.3756
      5        0.6905        0.6898  0.3922
      6        0.6877        0.6891  0.3916
      7        0.6911        0.6888  0.3741
      8        0.6862        0.6874  0.3741
      9        0.6824        0.7113  0.3744
     10        0.6762        0.7054  0.3761
     11        0.6391        0.6050  0.3754
     12        0.5695        0.5648  0.3737
     13        0.5285        0.5401  0.3743
     14        0.5088        0.5711  0.3753
     15        0.4885        0.5317  0.3755
     16        0.4728        0.5089  0.3731
     17        0.4566        0.4966  0.3734
     18        0.4433        0.5376  0.3731
     19        0.4517        0.5155  0.3864
     20        0.4118        0.4916  0.3760
     21        0.4103        0.4

     16        0.5933        0.5601  0.3819
     17        0.5866        0.5539  0.3740
     18        0.5796        0.5471  0.3735
     19        0.5724        0.5396  0.3730
     20        0.5648        0.5320  0.3718
     21        0.5570        0.5239  0.3727
     22        0.5496        0.5166  0.3719
     23        0.5413        0.5086  0.3705
     24        0.5321        0.5017  0.3722
     25        0.5223        0.4962  0.3722
     26        0.5123        0.4911  0.3716
     27        0.5015        0.4845  0.3707
     28        0.4907        0.4758  0.3788
     29        0.4788        0.4652  0.3758
     30        0.4667        0.4528  0.3712
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.6915        0.6885  0.3707
      2        0.6906        0.6868  0.3723
      3        0.6900        0.6856  0.3712
      4        0.6895        0.6844  0.3715
      5        0.6886        0.6829  0.3721
      6        0.6874        0.6

     23        0.6886        0.6849  0.3774
     24        0.6886        0.6848  0.3804
     25        0.6885        0.6846  0.3765
     26        0.6884        0.6844  0.3816
     27        0.6882        0.6842  0.3773
     28        0.6881        0.6840  0.3813
     29        0.6880        0.6838  0.3798
     30        0.6878        0.6836  0.3888
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.6906        0.6933  0.3532
      2        0.6905        0.6905  0.3528
      3        0.6867        0.6894  0.3468
      4        0.6825        0.6875  0.3444
      5        0.6811        0.6955  0.3437
      6        0.6747        0.6778  0.3436
      7        0.6538        0.6678  0.3445
      8        0.6021        0.5668  0.3435
      9        0.5352        0.5108  0.3454
     10        0.4920        0.4845  0.3442
     11        0.4738        0.4700  0.3444
     12        0.4713        0.4802  0.3455
     13        0.4457        0.4

      6        0.6822        0.6749  0.3692
      7        0.6794        0.6700  0.4069
      8        0.6757        0.6636  0.3406
      9        0.6712        0.6555  0.3456
     10        0.6657        0.6463  0.3409
     11        0.6600        0.6367  0.3402
     12        0.6541        0.6275  0.3405
     13        0.6483        0.6191  0.3400
     14        0.6424        0.6114  0.3393
     15        0.6366        0.6042  0.3399
     16        0.6306        0.5976  0.3391
     17        0.6247        0.5908  0.3472
     18        0.6188        0.5831  0.3404
     19        0.6125        0.5751  0.3394
     20        0.6060        0.5660  0.3383
     21        0.5993        0.5576  0.3390
     22        0.5929        0.5488  0.3417
     23        0.5862        0.5413  0.3355
     24        0.5797        0.5331  0.3382
     25        0.5728        0.5271  0.3416
     26        0.5668        0.5201  0.3399
     27        0.5601        0.5139  0.3408
     28        0.5534        0.5

     13        0.6915        0.6904  0.3390
     14        0.6912        0.6900  0.3398
     15        0.6910        0.6896  0.3397
     16        0.6908        0.6892  0.3420
     17        0.6906        0.6889  0.3425
     18        0.6904        0.6886  0.3392
     19        0.6902        0.6883  0.3399
     20        0.6901        0.6881  0.3385
     21        0.6899        0.6878  0.3379
     22        0.6898        0.6875  0.3399
     23        0.6896        0.6873  0.3391
     24        0.6895        0.6870  0.3388
     25        0.6893        0.6867  0.3411
     26        0.6892        0.6864  0.3397
     27        0.6890        0.6862  0.3389
     28        0.6889        0.6859  0.3399
     29        0.6887        0.6857  0.3398
     30        0.6886        0.6854  0.3385
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.6948        0.6953  0.5204
      2        0.6939        0.6941  0.5274
      3        0.6930        0.6

GridSearchCV(cv=None, error_score='raise',
       estimator=<class 'gcnn.utils.NNplusplus'>[uninitialized](
  module=<class 'gcnn.nets.BaselineCNNSimple'>,
),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'batch_size': [25, 50], 'lr': [0.001, 0.0001, 1e-05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
random_search_cnn.best

In [30]:
import scipy.sparse
from scipy.sparse import csgraph
from gcnn.nets import GraphCNNSimple
from gcnn.utils import NNplusplus

In [31]:
grid = graph.knn(graph.grid_coordinates(75), k=8, metric='cityblock')
#grid = nx.adjacency_matrix(graph, sorted(graph.nodes))

In [32]:
def coarsen_permute(graph, bands):
    
    # node should be ordered in a way corresponding to bands data order
    grid = nx.adjacency_matrix(graph, sorted(graph.nodes))
    
    # coarsen grid
    grid = sp.sparse.csr.csr_matrix(grid)
    laps, perms = graclus.coarsen(grid, levels=3, self_connections=False)
    print()
    
    # compute normed laplacian
    laps = [csgraph.laplacian(g, normed=True) for g in laps[:-1]]
    
    # update data accordingly
    pbands = graclus.perm_data(bands, perms).astype(np.float32)
    
    return laps, pbands

In [33]:
laps, pbands = coarsen_permute(grid, bands[:, 0, :, :].reshape(-1, 75 * 75))

Layer 0: M_0 = |V| = 6264 nodes (639 added),|E| = 33018 edges
Layer 1: M_1 = |V| = 3132 nodes (211 added),|E| = 17503 edges
Layer 2: M_2 = |V| = 1566 nodes (55 added),|E| = 7751 edges
Layer 3: M_3 = |V| = 783 nodes (0 added),|E| = 3289 edges



In [34]:
param_dist_cnn = {
    'batch_size': [25,50],
    'lr':[0.001,0.0001,0.00001],
}
n_iter_search_cnn = 6

In [35]:
l0 = torch.from_numpy(laps[0].todense()).float()
l2 = torch.from_numpy(laps[2].todense()).float()

if cuda:
    l0 = l0.cuda()
    l2 = l2.cuda()

net = NNplusplus(
    GraphCNNSimple,
    module__k=25,
    module__lf0=l0,
    module__lf2=l2,
    use_cuda=cuda,
    optimizer=torch.optim.Adam,
    criterion=nn.BCEWithLogitsLoss, 
    batch_size=50,
    max_epochs=15,
    lr=0.001,
)

In [36]:
features = pbands[train]
labels = targets[train].astype(np.float32)

In [37]:
random_search_net = GridSearchCV(net,param_dist_cnn)

In [38]:
features.shape

(1363, 6264)

In [39]:
random_search_net.fit(features,labels)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.7294        0.7030  5.4334
      2        0.6849        0.7388  5.4315
      3        0.6831        0.6652  5.4292
      4        0.6648        0.6672  5.4379
      5        0.6702        0.6616  5.4448
      6        0.6604        0.6670  5.4370
      7        0.6664        0.6661  5.4363
      8        0.6593        0.6626  5.4294
      9        0.6585        0.6553  5.4330
     10        0.6568        0.6505  5.4328
     11        0.6510        0.6429  5.4296
     12        0.6536        0.6883  5.4274
     13        0.6533        0.6761  5.4318
     14        0.6629        0.6542  5.4309
     15        0.6510        0.6562  5.4326
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.7036        0.6822  5.4389
      2        0.6850        0.6924  5.4349
      3        0.6910        0.6864  5.4368
      4        0.6912        0.6

      9        0.6869        0.6810  5.4378
     10        0.6861        0.6799  5.4424
     11        0.6852        0.6785  5.4370
     12        0.6840        0.6769  5.4400
     13        0.6827        0.6752  5.4401
     14        0.6815        0.6736  5.4325
     15        0.6799        0.6714  5.4316
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.7042        0.6810  3.6191
      2        0.6731        0.6705  3.6251


RuntimeError: cuda runtime error (2) : out of memory at /pytorch/torch/lib/THC/generic/THCStorage.cu:58